In [10]:
import json
import datasets
import abctk.obj.comparative as aoc

In [11]:
# Load previous dataset from the HF repo
dataset_raw = datasets.load_dataset(
    "abctreebank/comparative-NER-BCCWJ",
    use_auth_token = True,
    revision = "de8fe2785391897efc898be36d695d8164863045",
)

# Collect test sentence IDs
dataset_test = dataset_raw["test"]
test_IDs = set(record["ID"] for record in dataset_test)

Using custom data configuration abctreebank--comparative-NER-BCCWJ-c32c3cdce4ba824a
Found cached dataset parquet (/home/owner/.cache/huggingface/datasets/abctreebank___parquet/abctreebank--comparative-NER-BCCWJ-c32c3cdce4ba824a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# Collect comments put on trees
# First, parse trees with amoove
# cat *.psd | ../amoove/bin/abc-comp-conv abc2jsonl > from_tree.jsonl
trees = []

with open("../from_tree.jsonl") as f_tree:
    for line in filter(None, map(str.strip, f_tree)):
        trees.append(json.loads(line))

In [4]:
# Load the latest data
with open("../../comp-proto/comparative-annotation_linearilized_2022-12-27.txt") as g:
    records = tuple(
        aoc.read_bracket_annotation_file(g)
    )

In [5]:
# Merge comments
for rec in records:
    for tr in trees:
        if tr["ID"] == rec["ID"]:
            tr_comments = tr.get("comments", tuple())
            if "comments" in rec:
                rec["comments"].extend(tr_comments)
            else:
                rec["comments"] = list(tr_comments)

In [6]:
# Split train/test
records_train = tuple(
    r for r in records
    if r["ID"] not in test_IDs
)

records_test = tuple(
    r for r in records
    if r["ID"] in test_IDs
)

In [9]:
with open("train.jsonl", "w") as f_train:
    for record in records_train:
        json.dump(record, f_train)
        f_train.write("\n")

with open("test.jsonl", "w") as f_test:
    for record in records_test:
        json.dump(record, f_test)
        f_test.write("\n")


Put the generated two JSONL files to the dataset folder and do HF push_to_hub.

```sh
mv train.sjon test.jsonl ../comparative-NER-BCCWJ/
abct-comp-ner-utils upload-data --private ./comparative-NER-BCCWJ
```

WARNING: do not stage any annotation data to this repo so as to keep them private.